In [1]:
root_dir = '/home/xoel/Escritorio/mastersthesis/'
data_dir = root_dir+'data/'
lists_dir = data_dir+'lists/'
scripts_dir = root_dir+'scripts/'
results_dir = root_dir+'results/'
plots_dir = root_dir+'plots/'

import pandas as pd
import numpy as np
from timeit import timeit
import sys
sys.path.insert(1, '/home/xoel/Escritorio/pyMKT/rebase/')
import MKT

#################################################################################
pops=['EUR']
tests=['eMKT']
thresholds=[[0.15]]
#################################################################################


#################################################################################
genes = pd.read_csv(lists_dir+'exp_aa.csv', header=[0,1], index_col=0)
ph = pd.read_csv(data_dir+'metaPops.tsv', sep='\t')
#################################################################################


#################################################################################
# Create dataframe with the group metadata
groups = pd.DataFrame(np.array([[x[0], x[1]] for x in genes.columns.values]), columns=['Temporal', 'Anatomical'])

# Accomodate header
genes.columns = list(map(lambda x: '_'.join(x), genes.columns.values))

# Relate to samples
groups.index=genes.columns
#################################################################################


/home/xoel/anaconda3/lib/python3.7/site-packages/numba/core/cpu.py:77: UserWarning: Numba extension module 'numba_scipy' failed to load due to 'ValueError(No function '__pyx_fuse_0pdtr' found in __pyx_capi__ of 'scipy.special.cython_special')'.
  numba.core.entrypoints.init_all()


In [ ]:
def gral_perm(df, ph, pops=['EUR'], tests=['eMKT'], thresholds=[[0.15]], reps=100):
    
    # Dictionary of results
    results = {}

    # Compute original estimations
    results['original'] = MKT.MKT(df, ph).test(populations=pops, thresholds=thresholds, tests=tests)

    # Create result dictionary of general permutation test
    results['GRAL'] = {}

    # Store generally constant and variable genes
    gral_ct = df[(df==1).all(1)].iloc[:,[0]]
    gral_ct.columns = ['constant']
    gral_vars = df[~(df.sum(1)==0) & (~((df==1).all(1)))]

    # Store general constant and variable genes
    results['GRAL']['observed']['constant'] = MKT.MKT(gral_ct, ph).test(populations=pops, thresholds=thresholds, tests=tests)
    results['GRAL']['observed']['variables'] = MKT.MKT(gral_vars, ph).test(populations=pops, thresholds=thresholds, tests=tests)
    
    
    


In [2]:
def permute_df(df, nr):
    all_dfs = []
    for _ in range(nr):
        iter_df = df.copy()
        iter_df.index = np.random.permutation(df.index)
        all_dfs.append(iter_df)
    return all_dfs
        

In [5]:
MKT.MKT(pd.concat(permute_df(df, 100), 1), ph).test(populations=pops, thresholds=thresholds, tests=tests)


· [1/2] Parsing population data [DONE]
· [2/2] Running tests [DONE]


,Ka,Ks,omega,alpha,neg_b,neg_f,neg_d,pvalue,omegaA,omegaD,name,population,test,threshold,ngenes
0,0.004176,0.020175,0.206989,0.062768,0.169749,0.193996,0.636255,0.000043,0.012992,0.193996,W1_OFC,EUR,eMKT,0.15,6721
1,0.004149,0.020239,0.205022,0.050509,0.169127,0.194667,0.636207,0.000045,0.010356,0.194667,W1_DFC,EUR,eMKT,0.15,6462
2,0.004134,0.020168,0.204958,0.056763,0.168490,0.193324,0.638187,0.000040,0.011634,0.193324,W1_VFC,EUR,eMKT,0.15,5598
3,0.004181,0.020278,0.206177,0.055653,0.168370,0.194702,0.636928,0.000042,0.011474,0.194702,W1_MFC,EUR,eMKT,0.15,6357
4,0.004155,0.020176,0.205942,0.038864,0.165803,0.197939,0.636259,0.000533,0.008004,0.197939,W1_M1C,EUR,eMKT,0.15,6157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16995,0.004210,0.019980,0.210692,0.061892,0.166670,0.197652,0.635678,0.000052,0.013040,0.197652,ALL_AMY,EUR,eMKT,0.15,9682
16996,0.004195,0.020073,0.208981,0.064091,0.167808,0.195587,0.636605,0.000052,0.013394,0.195587,ALL_STR,EUR,eMKT,0.15,10043
16997,0.004218,0.020143,0.209416,0.062748,0.167117,0.196275,0.636608,0.000053,0.013140,0.196275,ALL_MD,EUR,eMKT,0.15,10072
16998,0.004160,0.019916,0.208900,0.073673,0.169086,0.193510,0.637404,0.000051,0.015390,0.193510,ALL_CBC,EUR,eMKT,0.15,9489


In [4]:
df = genes
mixed = True
vars_only = True

In [83]:


# Store unique values of variables
uniques = {}

# Iterate variables
for var in groups.columns:
    
    # Inside the dictionary of the variable, the key will be each distint value of the variable
    ct_sets[var] = {}
    var_sets[var] = {}
    
    # Save unique values of the variable
    uniques[var] = groups[var].unique()
    
    # Iterate over values
    for value in uniques[var]:
        # Select matching samples
        selection = groups[groups[var]==value].index
        iter_df = df[selection]
        # Store constant and variable gene dfs
        ct_sets[var][value] = iter_df[(iter_df==1).all(1)]
        var_sets[var][value] = iter_df[~(iter_df.sum(1)==0) & (~((iter_df==1).all(1)))]

# Combine variables to check for effect of interacting variables
if mixed:
    mix_uniques = {}
    if len(groups.columns) != 2: # Will generalize for more vars later on...
        print('Mixed effects only available for ')
    var1, var2 = uniques.keys()
    mix_uniques[f'{var1}_{var2}'] = [(x,y) for x in uniques[var1] for y in uniques[var2]]


In [82]:
for sample in groups.index:
    for var in uniques.keys()

W1_OFC
W1_DFC
W1_VFC
W1_MFC
W1_M1C
W1_S1C
W1_IPC
W1_A1C
W1_STC
W1_ITC
W1_V1C
W1_HIP
W1_AMY
W1_STR
W1_MD
W1_CBC
W1_ALL
W2_OFC
W2_DFC
W2_VFC
W2_MFC
W2_M1C
W2_S1C
W2_IPC
W2_A1C
W2_STC
W2_ITC
W2_V1C
W2_HIP
W2_AMY
W2_STR
W2_MD
W2_CBC
W2_ALL
W3_OFC
W3_DFC
W3_VFC
W3_MFC
W3_M1C
W3_S1C
W3_IPC
W3_A1C
W3_STC
W3_ITC
W3_V1C
W3_HIP
W3_AMY
W3_STR
W3_MD
W3_CBC
W3_ALL
W4_OFC
W4_DFC
W4_VFC
W4_MFC
W4_M1C
W4_S1C
W4_IPC
W4_A1C
W4_STC
W4_ITC
W4_V1C
W4_HIP
W4_AMY
W4_STR
W4_MD
W4_CBC
W4_ALL
W5_OFC
W5_DFC
W5_VFC
W5_MFC
W5_M1C
W5_S1C
W5_IPC
W5_A1C
W5_STC
W5_ITC
W5_V1C
W5_HIP
W5_AMY
W5_STR
W5_MD
W5_CBC
W5_ALL
W6_OFC
W6_DFC
W6_VFC
W6_MFC
W6_M1C
W6_S1C
W6_IPC
W6_A1C
W6_STC
W6_ITC
W6_V1C
W6_HIP
W6_AMY
W6_STR
W6_MD
W6_CBC
W6_ALL
W7_OFC
W7_DFC
W7_VFC
W7_MFC
W7_M1C
W7_S1C
W7_IPC
W7_A1C
W7_STC
W7_ITC
W7_V1C
W7_HIP
W7_AMY
W7_STR
W7_MD
W7_CBC
W7_ALL
W8_OFC
W8_DFC
W8_VFC
W8_MFC
W8_M1C
W8_S1C
W8_IPC
W8_A1C
W8_STC
W8_ITC
W8_V1C
W8_HIP
W8_AMY
W8_STR
W8_MD
W8_CBC
W8_ALL
W9_OFC
W9_DFC
W9_VFC
W9_MFC
W9_M1C
W9_S1C
W9_IPC
W9_A1C


In [8]:
# Per-stage list of genes
stg_dict = {}
for stg in stg_list:
    stg_dict[stg] = set(eq_dfs['aa'][stg][~(eq_dfs['aa'][stg]==0).all(axis=1) &
                                    ~(eq_dfs['aa'][stg]==1).all(axis=1)].index.values)
# Per-region list of genes
reg_dict = {}
for reg in reg_list:
    reg_dict[reg] = set(eq_dfs['aa'].xs(reg, level=1, drop_level=False, axis=1)[~(eq_dfs['aa'].xs(reg, level=1, drop_level=False, axis=1) ==0).all(axis=1) &
                                    ~(eq_dfs['aa'].xs(reg, level=1, drop_level=False, axis=1) ==1).all(axis=1)].index.values)


# Analisys of column normalizing per region and stage
def percol(col, reps=1000):
    stg, reg = col.name
    print(stg, reg, end=' ')
    genes = stg_dict[stg].intersection(reg_dict[reg])
    iter_col = col[genes].copy()
    
    # Point results
    result = mkt_on_col(iter_col, ph_df, pops=pops, tests=tests, cutoffs=cutoffs, do_trims=do_trims)
    result['label'] = 'aa'
    
    # Permutation results
    perm_list = []
    for j in range(reps):
        iter_col.index = np.random.permutation(iter_col.index.values)
        perm_list.append(mkt_on_col(iter_col, ph_df, pops=pops, tests=tests, cutoffs=cutoffs, do_trims=do_trims))
    perm_results = pd.concat(perm_list, ignore_index=True, axis=0)
    perm_results['label'] = 'aa_perm'
    perm_results['nullgenes'] = len(iter_col.index.values)
    return result, perm_results

NameError: name 'eq_dfs' is not defined